<a href="https://colab.research.google.com/github/perv0vser/Test-Projects/blob/main/%D0%A2%D0%B5%D1%81%D1%82%D0%BE%D0%B2%D0%BE%D0%B5_%D0%BD%D0%B0_sql_%D0%B8_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1: Выявление аномалий в данных о посещениях сайта

У вас есть данные о посещениях сайта за последние 10 дней. Необходимо выявить аномалии в количестве уникальных пользователей.

Проведите анализ данных:

- Определите, есть ли выбросы в количестве уникальных посетителей сайта.
- Если есть, укажите их и предложите возможные причины появления таких выбросов.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data = {
    'date': ['2024-10-01', '2024-10-02', '2024-10-03', '2024-10-04', '2024-10-05',
             '2024-10-06', '2024-10-07', '2024-10-08', '2024-10-09', '2024-10-10'],
    'unique_visitors': [200, 220, 250, 240, 10000, 230, 245, 240, 235, 50]
}
df1 = pd.DataFrame(data)

# Определение выбросов с использованием IQR
Q1 = df1['unique_visitors'].quantile(0.25)
Q3 = df1['unique_visitors'].quantile(0.75)
IQR = Q3 - Q1

# Определение границ для выбросов
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Нахождение аномалий
outliers = df1[(df1['unique_visitors'] < lower_bound) | (df1['unique_visitors'] > upper_bound)]

# Вывод результатов
print("Выбросы:\n", outliers)

Выбросы:
          date  unique_visitors
4  2024-10-05            10000
9  2024-10-10               50


**Комментарий:** Выбросы есть. Они выведены выше. Возможные причины могут быть разнообразными, например технический, продуктовый баг или баг в аналитике, еще пиковая аномалия в "униках" пришлась на субботу и это единственная суббота, попавшая в наш временной ряд, поэтому невозможно рассуждать о сезонности, но если она и есть, то вряд ли настолько резкая. Еще нужно посмотреть в сторону каких-то вирусных инфоповодов, которые в этот день могли привести трафик, ну и конечно узнать, были ли какие-то изменения в продукте (запускались или заканчивались рекламные компании, переделали флоу, выпилили фичу, раскатили вариант АБ-теста на 100% и т.п.). Не стоит сбрасывать со счетов и возможные злонамеренные действия (ботов). А вообще, на такие случаи неплохо иметь автоматизированную систему оповещений (выявить критические метрики и пороговые значения для них, сделать бота, отслеживающего аномалии во временных рядах с этими метриками, который через определенный промежуток времени проверяет аномалии и сигнализирует, завести дашборд по аномалиям).

# Задание 2: SQL запрос для анализа активности пользователей

У вас есть таблица сессий пользователей на сайте, которая называется sessions

Напишите SQL-запрос, который вернет:

- Среднюю продолжительность сессии для каждого пользователя.
- Пользователей, которые провели на сайте больше 100 минут за день.

In [ ]:
import sqlite3

In [ ]:
conn = sqlite3.connect('df.db')

In [ ]:
df2 = pd.DataFrame()

In [ ]:
df2['session_id'] = list(map(lambda num: str(num), range(1, 11)))
df2['user_id'] = list(map(lambda num: str(num), range(100, 105)))*2
df2['session_start'] = ['2024-08-01 10:00', '2024-08-01 11:00', '2024-08-01 12:00', '2024-08-01 13:00',
                      '2024-08-01 14:00', '2024-08-01 15:00', '2024-08-01 16:00', '2024-08-01 17:00',
                      '2024-08-01 18:00', '2024-08-01 19:00']
df2['session_end'] = ['2024-08-01 10:45', '2024-08-01 11:30', '2024-08-01 12:15', '2024-08-01 13:10',
                      '2024-08-01 14:50', '2024-08-01 15:55', '2024-08-01 16:25', '2024-08-01 17:47',
                      '2024-08-01 18:40', '2024-08-01 19:55']

In [ ]:
df2

,session_id,user_id,session_start,session_end
0,1,100,2024-08-01 10:00,2024-08-01 10:45
1,2,101,2024-08-01 11:00,2024-08-01 11:30
2,3,102,2024-08-01 12:00,2024-08-01 12:15
3,4,103,2024-08-01 13:00,2024-08-01 13:10
4,5,104,2024-08-01 14:00,2024-08-01 14:50
5,6,100,2024-08-01 15:00,2024-08-01 15:55
6,7,101,2024-08-01 16:00,2024-08-01 16:25
7,8,102,2024-08-01 17:00,2024-08-01 17:47
8,9,103,2024-08-01 18:00,2024-08-01 18:40
9,10,104,2024-08-01 19:00,2024-08-01 19:55


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   session_id     10 non-null     object
 1   user_id        10 non-null     object
 2   session_start  10 non-null     object
 3   session_end    10 non-null     object
dtypes: object(4)
memory usage: 448.0+ bytes


In [ ]:
df2['session_start'] = pd.to_datetime(df2['session_start'])
df2['session_end'] = pd.to_datetime(df2['session_end'])

In [ ]:
df2['session_duration'] = df2['session_end'] - df2['session_start']

In [ ]:
df2['session_duration'] = df2['session_duration'].dt.total_seconds() / 60

In [ ]:
df2['session_duration'] = df2['session_duration'].astype('int')

In [ ]:
df2

,session_id,user_id,session_start,session_end,session_duration
0,1,100,2024-08-01 10:00:00,2024-08-01 10:45:00,45
1,2,101,2024-08-01 11:00:00,2024-08-01 11:30:00,30
2,3,102,2024-08-01 12:00:00,2024-08-01 12:15:00,15
3,4,103,2024-08-01 13:00:00,2024-08-01 13:10:00,10
4,5,104,2024-08-01 14:00:00,2024-08-01 14:50:00,50
5,6,100,2024-08-01 15:00:00,2024-08-01 15:55:00,55
6,7,101,2024-08-01 16:00:00,2024-08-01 16:25:00,25
7,8,102,2024-08-01 17:00:00,2024-08-01 17:47:00,47
8,9,103,2024-08-01 18:00:00,2024-08-01 18:40:00,40
9,10,104,2024-08-01 19:00:00,2024-08-01 19:55:00,55


In [ ]:
 df2.to_sql('test2', conn, if_exists='replace', index=False)

10

In [ ]:
sql1 = '''SELECT user_id,
                 AVG(session_duration)
          FROM test2
          GROUP BY user_id'''

In [ ]:
sql2 = '''SELECT user_id,
                 SUM(session_duration) AS daily_session
          FROM test2
          GROUP BY user_id
          HAVING daily_session > 100'''

In [ ]:
sql_requests = [sql1, sql2]

In [ ]:
def db_reply(request_list):
    '''Возвращает результаты списка SQL запросов к базе данных'''
    for index, request in enumerate(request_list):
        print(f'Ответ по запросу {index}:')
        display(pd.read_sql(request, conn))
        print('**************************')

In [ ]:
db_reply(sql_requests)

Ответ по запросу 0:


,user_id,AVG(session_duration)
0,100,50.0
1,101,27.5
2,102,31.0
3,103,25.0
4,104,52.5


**************************
Ответ по запросу 1:


,user_id,daily_session
0,104,105


**************************


**Комментарий:** Выводы запросов см. выше. Сами запросы реализовал через библиотеку Python для работы с SQL и написание функции. Если запросов несколько удобно запустить их одновременно.

# Задание 3: Аномалия в охвате сайта

Вы видите, что у сайта был аномально высокий охват в день X (например, посещения выросли в 5 раз по сравнению с предыдущими днями). Ваша задача — предложить план действий для выяснения причин.

Задача:

1.   Опишите, какие шаги вы предпримете для анализа ситуации?
2.   Какие инструменты и подходы вы будете использовать для проверки данных?

**Ответ:**
1. Шаги для анализа ситуации:

- Сравнить данные за день X с данными за предыдущие периоды (не только дни, но неделю, месяц, несколько месяцев). Проанализировать не только охват, но и другие метрики, такие как количество посещений, уникальные пользователи, продолжительность сессии, количество просмотренных страниц и показатели отказов, чтобы сравнить сценарии поведения пользователей.
- Проанализировать источник посещений и логи сайта (это поможет понять, являются ли переходы реальными или это накрутка ботами). Трафик из определённого региона, возрастание числа показателей отказов, сомнительные страницы входа - все это может быть "звоночками".
- Изучить прямой трафик. Резкие всплески, когда прямой трафик в несколько раз превышает поисковый, могут быть признаками накрутки посещаемости ботами.
- Проверить рекламные кампании (запуск, остановка, значительные изменения в расходах на рекламу).
- Изучить, был ли в этот день опубликован уникальный  или вирусный контент, который мог привлечь охваты.

2. Инструменты для проверки данных:

- Инструменты веб-аналитики (например Яндекс Метрика, Google Analytics);
- Платформы аналитики социальных сетей помогут выяснить, были ли посты, достигшие большого охвата;
- Инструменты для анализа контента помогут выяснить, какой контент был наиболее популярным и вирусным в социальных медиа.

# Задание 4: Обработка данных о приложении

Представьте, что вы работаете с данными о сессиях мобильного приложения.

Используя PySpark (или Python, если с PySpark нет опыта), напишите код, который:
- Считает количество уникальных пользователей для каждой платформы (iOS, Android).
- Определяет среднюю продолжительность сессии для каждой платформы.

In [ ]:
df4 = pd.DataFrame()

In [ ]:
df4['session_id'] = list(map(lambda num: str(num), range(1, 11)))
df4['user_id'] = list(map(lambda num: str(num), range(100, 106))) + list(map(lambda num: str(num), range(100, 104)))
df4['platform'] = ['iOS', 'Android']*5
df4['session_start'] = ['2024-08-01 10:00', '2024-08-01 11:00', '2024-08-01 12:00', '2024-08-01 13:00',
                      '2024-08-01 14:00', '2024-08-01 15:00', '2024-08-01 16:00', '2024-08-01 17:00',
                      '2024-08-01 18:00', '2024-08-01 19:00']
df4['session_end'] = ['2024-08-01 10:45', '2024-08-01 11:30', '2024-08-01 12:15', '2024-08-01 13:10',
                      '2024-08-01 14:50', '2024-08-01 15:55', '2024-08-01 16:25', '2024-08-01 17:47',
                      '2024-08-01 18:40', '2024-08-01 19:55']

In [ ]:
df4['session_start'] = pd.to_datetime(df4['session_start'])
df4['session_end'] = pd.to_datetime(df4['session_end'])

In [ ]:
df4['session_duration'] = df4['session_end'] - df4['session_start']
df4['session_duration'] = df4['session_duration'].dt.total_seconds() / 60
df4['session_duration'] = df4['session_duration'].astype('int')

In [ ]:
df4

,session_id,user_id,platform,session_start,session_end,session_duration
0,1,100,iOS,2024-08-01 10:00:00,2024-08-01 10:45:00,45
1,2,101,Android,2024-08-01 11:00:00,2024-08-01 11:30:00,30
2,3,102,iOS,2024-08-01 12:00:00,2024-08-01 12:15:00,15
3,4,103,Android,2024-08-01 13:00:00,2024-08-01 13:10:00,10
4,5,104,iOS,2024-08-01 14:00:00,2024-08-01 14:50:00,50
5,6,105,Android,2024-08-01 15:00:00,2024-08-01 15:55:00,55
6,7,100,iOS,2024-08-01 16:00:00,2024-08-01 16:25:00,25
7,8,101,Android,2024-08-01 17:00:00,2024-08-01 17:47:00,47
8,9,102,iOS,2024-08-01 18:00:00,2024-08-01 18:40:00,40
9,10,103,Android,2024-08-01 19:00:00,2024-08-01 19:55:00,55


In [ ]:
agg_func = {'user_id': ['nunique'], 'session_duration': ['mean']}
df4.groupby(['platform']).agg(agg_func).round(2)

,user_id,session_duration
,nunique,mean
platform,,
Android,3,39.4
iOS,3,35.0


**Комментарий:** Вывод результатов см. выше. В этом и первом задании, где было предпочтительно использовать Spark, но допускался Python, использовал последний, так как в нем больше уверен (больше опыта). Небольшой опыт в Spark был и если потребуется его применять для повседневных задач, думаю, смогу быстро набить руку.

# Задание 5: Аномалия в среднедневном времени посещения на пользователя

Вы видите, что среднедневное время пользователей на сайте Х за октябрь 2024 года упало с 15 до 3 минут. Ваша задача — предложить план действий для выяснения причин.

Среднее время пользователей за месяц считается как общее время всех пользователей на сайте за месяц, разделенное на количество дней и количество пользователей сайта за месяц.

1.   Опишите, какие шаги вы предпримете для анализа ситуации?
2.   Опишите, что может стать причиной такого снижения времени?
3.   Какие инструменты и подходы вы будете использовать для проверки данных?

Снижение среднего времени пользователей на сайте с 15 до 3 минут может сигнализировать о серьезных проблемах с контентом, пользовательским опытом или техническими аспектами сайта. Вот план действий для анализа ситуации, возможные причины и инструменты для проверки данных.

1. Шаги для анализа ситуации:
- Оценить, есть ли технические проблемы: загрузка страниц, ошибки сервера, низкая скорость загрузки.
- Выяснить, выкатывались ли какие-то новые фичи, кампании.
- Сравнить данные за октябрь 2024 года с данными за предыдущие месяцы и годы.
Изучить динамику среднего времени сессии, количества посетителей и показателей отказов. Декомпозировать метрику среднего времени и посмотреть, снизилось ли оно при том же количестве "уников" или уменьшилось за счет большого притока дополнительных пользователей, но с очень короткими сессиями, к примеру.
- Определить, снизилось ли время пользователей по всем источникам трафика или только по конкретным (например, органический, платный, реферальный).
- Определить, какие страницы имеют наибольшее снижение времени. Возможно, на этих страницах произошли изменения или они были заменены другими. Выяснить, могло ли удаление определенного контента повлиять на интерес пользователей.
- Провести опрос пользователей (или изучить жалобы/обратную связь), чтобы выяснить, что именно их не устраивает на сайте (дизайн, скорость загрузки, качество контента и т. д.).

2. Возможные причины снижения времени пользователей:
Технические проблемы, качество контента, изменение дизайна или функциональности сайта, обновления, которые ухудшили удобство навигации или представление информации, привлечение новой аудитории, которая хуже реагировать на контент,
сезонные изменения, конкуренция или изменения в алгоритмах поисковых систем.

3. Инструменты и подходы для проверки данных:
Аналитический инструмент для получения подробной информации о поведении пользователей, источниках трафика и анализе контента; инструмент для изучения изменения в видимости сайта, индексации, а также выявления возможных проблемы;
инструменты проверки скорости загрузки страниц; инструменты визуализации поведения пользователей на страницах; инструменты для сбора обратной связи от пользователей.